In [53]:
import random
vowels = 'aeiou'
consonants = 'bcdfghjklmnpqrstvwxyz'

# Size of the hand to be used for the game. Using random size for fun
handSize = random.randrange(7,11,1)

# Read and pre-filter words from the dictionary.
# Words longer than hand size are skipped
# convert to lower case for faster comparison
def loadWords(maxLen): #Loads dictionary and strips words that are longer than the length of the hand
    inFile = open('scrabble dict.txt', 'r')
    wordList = []
    for line in inFile:
        thisWord = line.strip().lower()
        if len(thisWord) > maxLen :
            continue
        wordList.append(thisWord)
    print("", len(wordList), "possible words found.")
    return wordList

# Use the sequence to profile the use of each letter. 
# Based on the profile assign high score to low frequency letters
# Pre create a dictonary for the score corresponding to each letter
def getScoreDict(sequence): #Assign a score to each letter based on it's frequency in the dictionary of valid words
    freq = {}
    maxFreq = 0
    for x in sequence:
        freq[x] = freq.get(x,0) + 1
        if maxFreq < freq[x]:
            maxFreq = freq[x]
    for x in freq.keys():
        freq[x] = 5 - int(freq[x]*4.5//maxFreq)
    return freq

# Check if the word provided can be created using the letters in the hand.
# This function also returns the modified hand by removing the used letters
# If the word cannot be created in the given hand, the original hand is returned.
def checkWordInHand(checkWord, hand): #Checks if all the letters in word are available in hand
    checkLetters = list(checkWord)
    remainingHand = list(hand)
    if len(checkLetters) > len(remainingHand):
        return hand
    for letter in checkLetters:
        foundLetter = False
        for handLetter in remainingHand:
            if handLetter == letter:
                remainingHand.remove(letter)
                foundLetter = True
                break
        if not foundLetter :
            return hand
    return remainingHand

# Check the dictonary for the validity of the given word
def checkWordInDict(checkWord, wordList): #Checks if the word entered is valid from the word dictionary
    for dictword in wordList:
        if checkWord == dictword:
            return True
    return False

# Generate a random hand of letters for a given size
# Remaining letters from the previous hand are re-used
# New letters are chosen at random
# The priority of vowels vs consonents can be altered using the defined constants
# Ensure that the resultant hand has at least one vowel
def dealHand(hand, handSize):
    oneVowel = False
    for c in hand:
        if c in vowels:
            oneVowel = True
    dealSize = handSize - len(hand)
    vowelPriority = 2                     #Increases the probability of getting a vowel in hand
    vowelRange = len(vowels)*vowelPriority
    consonantPriority = 1
    consonantRange = len(consonants)*consonantPriority
    tmpHand = []
    while (not oneVowel) or (len(tmpHand) == 0) : #Makes sure there is at least one vowel in hand
        tmpHand = []
        for i in range(dealSize):
            pickNum = random.randrange(vowelRange + consonantRange)
            if pickNum < vowelRange:
                tmpHand.append(vowels[pickNum % len(vowels)])
                oneVowel = True 
            else:
                pickNum = pickNum - vowelRange
                tmpHand.append(consonants[pickNum % len(consonants)])
    return (hand + tmpHand)

# Calculate the score for creating a word
def calculatePoints(word, scoreDict):
    points = 0
    chars = list(word)
    for char in chars:
        points += scoreDict[char]
    return points

# Main flow
# 1. Pre-load and pre-process dictonary
dictWords = loadWords(handSize)

# 2. Create the score dictonary using the previously loaded dictonary.
#    Print the score dictonary
emptyString = ""
scoreDict = getScoreDict(emptyString.join(dictWords))
print("These are the points for each letter.\n",scoreDict,"\n")

# Initialize hand, score and terminating condition
hand = []
score = 0
wantToPlayMore = True

# Play until terminating condition - user to enter '.' for the word
while wantToPlayMore:
    
# 3. Deal a hand. A new hand for the first time and re-use previous unused hand with partial new hand subsequently.
    hand = dealHand(hand, handSize)
    
# Ask user to create a word. loop if the word 
#   - Cannot be created using the given hand
#   - Is not in the dictonary
    wordNotValid = True
    while wordNotValid:
        print("Your hand ", hand)
        word = input("Enter word or enter '.' to exit: ")
        lowerWord = word.strip().lower()
        if lowerWord == ".":
            wantToPlayMore = False
            break
        newHand = checkWordInHand(lowerWord, hand)
        print("Remaining hand ", newHand)
        if len(newHand) == len(hand): #Check if the word can be created using the given hand. 
                                      #If the letters are not consumed, the word cannot be created.
            score -= 1 #Penalty on score
            print("Your hand does not have all the letters for this word. Point deducted! Your new score is ", score, "\nTry Again!\n")
            continue
        if checkWordInDict(lowerWord, dictWords) : #Check if the word is valid
            hand = newHand
            points = calculatePoints(lowerWord, scoreDict)
            score = score + points # Calculate and add score
            wordNotValid = False # Letters consumed. Re-deal and start next round
            print("Valid Word! New score is ", score, "\n")
        else : #Gives an alert if an invalid word is entered
            score -= 1 #Penalty on score
            print("Not a valid word. Point deducted! Your new score is ", score, "\nTry again!\n")
            print("Revert hand ")               

 190831 possible words found.
These are the points for each letter.
 {'a': 2, 'h': 5, 'e': 1, 'd': 4, 'i': 2, 'n': 3, 'g': 4, 's': 2, 'l': 3, 'r': 3, 'v': 5, 'k': 5, 'w': 5, 'o': 3, 'f': 5, 't': 3, 'b': 5, 'c': 4, 'u': 4, 'm': 4, 'p': 4, 'y': 5, 'x': 5, 'j': 5, 'z': 5, 'q': 5} 

Your hand  ['e', 'i', 'y', 'k', 'r', 'o', 'e', 'c', 'd', 'd']
Enter word or enter '.' to exit: fffjfhgkjgkghj
Remaining hand  ['e', 'i', 'y', 'k', 'r', 'o', 'e', 'c', 'd', 'd']
Your hand does not have all the letters for this word. Point deducted! Your new score is  -1 
Try Again!

Your hand  ['e', 'i', 'y', 'k', 'r', 'o', 'e', 'c', 'd', 'd']
Enter word or enter '.' to exit: jhjhk;
Remaining hand  ['e', 'i', 'y', 'k', 'r', 'o', 'e', 'c', 'd', 'd']
Your hand does not have all the letters for this word. Point deducted! Your new score is  -2 
Try Again!

Your hand  ['e', 'i', 'y', 'k', 'r', 'o', 'e', 'c', 'd', 'd']
Enter word or enter '.' to exit: ei
Remaining hand  ['y', 'k', 'r', 'o', 'e', 'c', 'd', 'd']
Not a v